In [ ]:
from pbrlib.full import *
import pickle

In [ ]:
r = 'gaussian.opticalSimulationResults'
run = sorted(os.listdir(r))[-1]
p = r+'/'+run
folder = f'{p}/true/lightsource-PointSource/hitObject-OpticalAbsorberGroup'

In [ ]:
results = []
for f in os.listdir(folder):
  with open(folder+'/'+f, 'rb') as _f:
    results.append(pickle.load(_f))

In [ ]:
points = concatenate([r['points'] for r in results])[:,:2]

In [ ]:
Hs, Xs, Ys, _ = hist2d(points[:,0], points[:,1], bins=100)
savefig(f'gaussian-histogram-N{points.shape[0]}.png')

In [ ]:
gaussian = lambda X, A, s, x0: A*exp(-(X-x0)**2/s**2)

In [ ]:
distance = 100
thetaSigma = sqrt(1e-4)

In [ ]:
for i, (X, Y) in enumerate(
            [ ( (Xs[1:]+Xs[:-1])/2, Hs[argmin(abs(Ys)),:] ),
              ( (Ys[1:]+Ys[:-1])/2, Hs[:,argmin(abs(Xs))] ) ]):
  
  io.plotXY(X, Y, marker='x')
  popt, _ = scipy.optimize.curve_fit(gaussian, X, Y, p0=(max(Y), 10, 0))
  Xlin = linspace(min(X), max(X), 300)
  Ylin = gaussian(Xlin, *popt)
  plot(Xlin, Ylin)
  plot(Xlin, gaussian(Xlin, max(Y), distance*thetaSigma, 0))
  print(f'found sigma: {abs(popt[1]):.3f}, theoretical sigma: {distance*thetaSigma:.3f}')
  savefig(f'gaussian-histogram-{"xy"[i]}-N{points.shape[0]}.png')